# Import required libraries

In [1]:
!pip install wget # to download data
!pip install spacy
!python -m spacy download en_core_web_sm
!pip install gensim
!pip install nltk
!pip install scikit-learn


Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
/bin/bash: line 1: python: command not found
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [2]:
%matplotlib inline
import numpy as np
import gensim
import matplotlib.pyplot as plt
import seaborn as sns
import wget
import spacy
from spacy.tokenizer import Tokenizer
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import gensim.downloader as api
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import ParameterGrid
import scipy.stats
import tqdm

from tqdm import tqdm
from nltk.corpus import stopwords
import nltk
import re
import os
from collections import defaultdict
import jsonlines

import zipfile

%matplotlib widget
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Implicit representations

1. Create an embedding for the words using the (cleaned) Mosaico dataset by fitting a model.
2. Replace WordNet senses for each word and then create embeddings as before.

We can use different models to create word/sense embeddings and log which one performs better.

# Preprocessing experiments

In [3]:
# Take all the lines in the file
lines = []
with jsonlines.open('data/sample_annotated_sentences/500000.jsonl') as reader:

    limit = 10
    for i, line in enumerate(reader):

        if i < limit:
            print(f'{i:3d} | {line}')

        lines.append(line)

  0 | {'text': 'In Hinduism , the 60th birthday of a man is called Sashti poorthi .', 'annotations': [{'token_span': [5, 6], 'label': 'birthday%1:28:00::'}, {'token_span': [8, 9], 'label': 'man%1:18:00::'}]}
  1 | {'text': 'In New Zealand , both Metal Box and Second Edition briefly entered the Top 50 Albums Chart .', 'annotations': [{'token_span': [6, 7], 'label': 'box%1:06:00::'}, {'token_span': [8, 9], 'label': 'second%5:00:00:ordinal:00'}, {'token_span': [11, 12], 'label': 'enter%2:33:00::'}, {'token_span': [13, 14], 'label': 'top%3:00:00::'}]}
  2 | {'text': 'Article with extensive bibliography Article about the book " L\'hymnaire d\'Adonis " of Jacques d\'Adelswärd - Fersen ( in Spanish ) Pictures of Villa Lysis today [ 1 ] [ 2 ]', 'annotations': [{'token_span': [0, 1], 'label': 'article%1:10:00::'}, {'token_span': [4, 5], 'label': 'article%1:10:00::'}, {'token_span': [7, 8], 'label': 'book%1:10:00::'}]}
  3 | {'text': 'The root cluster attached to the basal plate of the bulb is t

In [4]:
def get_num_digits(number):
    return len(str(abs(number)))

def get_token_and_index_strings(text):
    line_tokens = text.split()
    line_tokens_print = []
    line_tokens_ids_print = []
    for i, token in enumerate(line_tokens):
        if get_num_digits(i) > len(token):
            line_tokens_print.append((' ' * (get_num_digits(i) - len(token))) + token)
        else:
            line_tokens_print.append(token)
        line_tokens_ids_print.append((' '*(len(line_tokens[i]) - len(str(i)))) + f'{i}')
    return ' '.join(line_tokens_print), ' '.join(line_tokens_ids_print)

def pretty_print(line, print_indexes=False):
    
    # Print the text
    text = line['text']
    
    # Print the indexes
    tokens_string, indexes_string = get_token_and_index_strings(text)
    print(f'text: {tokens_string}')
    if print_indexes:
        print(f'      {"".join(indexes_string)}')

    # Print the annotations
    annotations = line['annotations']
    if len(annotations) > 0:
        print(f'annotations: {annotations[0]}')
        for i in range(1, len(annotations)):
            print(f"{' '*(len('annotations: '))}{annotations[i]}")


In [5]:
# sample_line_1 = lines[0].copy()
sample_line_1 = {
    'text': "In Hinduism , the 60th birthday of a man is called Sashti poorthi .",
    'annotations': [
        {'token_span': [5, 6], 'label': 'birthday%1:28:00::'},
        {'token_span': [8, 9], 'label': 'man%1:18:00::'}
    ]
}

# sample_line_2 = lines[5].copy()
sample_line_2 = {
    'text': "The new world of English words came out in 1658 and a dictionary of 40,000 words had been prepared in 1721 by Nathan Bailey , though none was as comprehensive in breadth or style as Johnson's .",
    'annotations': [
        {'token_span': [1, 2], 'label': 'world%1:14:01::'},
        {'token_span': [4, 5], 'label': 'word%1:10:00::'},
        {'token_span': [11, 12], 'label': 'dictionary%1:10:00::'},
        {'token_span': [14, 15], 'label': 'word%1:10:00::'},
        {'token_span': [32, 33], 'label': 'style%1:10:00::'}
    ]
}

# sample_line_3 = lines[35].copy()
sample_line_3 = {
    'text': "Even without the availability of either co-receptor ( even CCR5 ) , the virus can still invade cells if gp41 were to go through an alteration ( including its cytoplasmic tail ) that resulted in the independence of CD4 without the need of CCR5 and / or CXCR4 as a doorway .",
    'annotations': [
        {'token_span': [17, 18], 'label': 'cell%1:03:00::'},
        {'token_span': [30, 31], 'label': 'tail%1:05:00::'},
        {'token_span': [50, 51], 'label': 'doorway%1:06:00::'}
    ]
}

# sample_line_4 = lines[15].copy()
sample_line_4 = {
    'text': "Typically , NATO inert munitions are painted entirely in light blue and / or have the word \" INERT \" stenciled on them in prominent locations .[ citation needed ] IED ( barrel bomb , nail bomb , pipe bomb , pressure cooker bomb , fertilizer bomb , molotov cocktail )", 
    'annotations': [
        {"token_span": [16, 17], "label": "word%1:10:00::"}, 
        {"token_span": [35, 36], "label": "nail%1:06:00::"}, 
        {"token_span": [48, 49], "label": "cocktail%1:13:00::"}
    ]
}

# sample_line_5 = lines[51].copy()
sample_line_5 = {
    'text': "Later on 9 January , Samuel scored a last - minute winner in the 4\u20133 win versus Siena by beating goalkeeper Gianluca Curci with a left - footed shot .", 
    'annotations': [
        {"token_span": [10, 11], "label": "minute%1:28:00::"}, 
        {"token_span": [11, 12], "label": "winner%1:18:00::"}, 
        {"token_span": [15, 16], "label": "win%1:11:00::"}, 
        {"token_span": [27, 28], "label": "foot%2:38:00::"}
    ]
}

# sample_line_6 = lines[502].copy()
sample_line_6 = {
    'text': "In a 1960 piano performance in Cologne , he played Chopin , threw himself on the piano and rushed into the audience , attacking Cage and pianist David Tudor by cutting their clothes with scissors and dumping shampoo on their heads .", 
    'annotations': [
        {"token_span": [24, 25], "label": "cage%1:18:00::"}, 
        {"token_span": [32, 33], "label": "clothes%1:06:00::"}, 
        {"token_span": [40, 41], "label": "head%1:08:00::"}
    ]
}


sample_lines = [
    sample_line_1,
    sample_line_2,
    sample_line_3,
    sample_line_4,
    sample_line_5,
    sample_line_6,
    lines[709].copy(),
    lines[704].copy()
]

for sample_line in sample_lines:
    pretty_print(sample_line, print_indexes=True)
    print()

text: In Hinduism , the 60th birthday of a man is called Sashti poorthi  .
       0        1 2   3    4        5  6 7   8  9     10     11      12 13
annotations: {'token_span': [5, 6], 'label': 'birthday%1:28:00::'}
             {'token_span': [8, 9], 'label': 'man%1:18:00::'}

text: The new world of English words came out in 1658 and  a dictionary of 40,000 words had been prepared in 1721 by Nathan Bailey  , though none was as comprehensive in breadth or style as Johnson's  .
        0   1     2  3       4     5    6   7  8    9  10 11         12 13     14    15  16   17       18 19   20 21     22     23 24     25   26  27 28            29 30      31 32    33 34        35 36
annotations: {'token_span': [1, 2], 'label': 'world%1:14:01::'}
             {'token_span': [4, 5], 'label': 'word%1:10:00::'}
             {'token_span': [11, 12], 'label': 'dictionary%1:10:00::'}
             {'token_span': [14, 15], 'label': 'word%1:10:00::'}
             {'token_span': [32, 33], 'label': 'sty

In [6]:
import unicodedata

def remove_accents_from_string(input_string):
    """
    Replace accented characters in a string with their plain versions using unicodedata.
    """
    normalized_string = unicodedata.normalize('NFKD', input_string)
    return ''.join([c for c in normalized_string if not unicodedata.combining(c)])

# Example usage:
original_string = "Café français"
result = remove_accents_from_string(original_string)
print(result)


Cafe francais


In [7]:
import stanza

nlp = stanza.Pipeline(lang='en', processors='tokenize,lemma', tokenize_pretokenized=True)

2023-11-19 10:24:47 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2023-11-19 10:24:49 INFO: Loading these models for language: en (English):
| Processor | Package           |
---------------------------------
| tokenize  | combined          |
| lemma     | combined_nocharlm |

2023-11-19 10:24:49 INFO: Using device: cpu
2023-11-19 10:24:49 INFO: Loading: tokenize
2023-11-19 10:24:49 INFO: Loading: lemma
2023-11-19 10:24:54 INFO: Done loading processors!


Tokenize with python splitting into sentences. This way we can use stanza to lemmatize the sentence pretokenizing the input

In [8]:
import nltk

# Download the punkt tokenizer if not already downloaded
nltk.download('punkt')

# Example string with multiple sentences
input_string = "This is the first sentence. Here is the second one. And finally, the third sentence!"

# Tokenize sentences and words using nltk
sentences = nltk.sent_tokenize(input_string)
list_of_lists = [nltk.word_tokenize(sentence) for sentence in sentences]

# Print the result
print(list_of_lists)


[['This', 'is', 'the', 'first', 'sentence', '.'], ['Here', 'is', 'the', 'second', 'one', '.'], ['And', 'finally', ',', 'the', 'third', 'sentence', '!']]


[nltk_data] Downloading package punkt to /home/daniel/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


We can implement the same thing simply with a split in our case

In [9]:
for sample_line in sample_lines:

    text = sample_line['text']
    text_tokens = [sentence.split() + ['.'] for sentence in text.split('.') if sentence]
    print(text_tokens)
    pretty_print(sample_line, print_indexes=True)


[['In', 'Hinduism', ',', 'the', '60th', 'birthday', 'of', 'a', 'man', 'is', 'called', 'Sashti', 'poorthi', '.']]
text: In Hinduism , the 60th birthday of a man is called Sashti poorthi  .
       0        1 2   3    4        5  6 7   8  9     10     11      12 13
annotations: {'token_span': [5, 6], 'label': 'birthday%1:28:00::'}
             {'token_span': [8, 9], 'label': 'man%1:18:00::'}
[['The', 'new', 'world', 'of', 'English', 'words', 'came', 'out', 'in', '1658', 'and', 'a', 'dictionary', 'of', '40,000', 'words', 'had', 'been', 'prepared', 'in', '1721', 'by', 'Nathan', 'Bailey', ',', 'though', 'none', 'was', 'as', 'comprehensive', 'in', 'breadth', 'or', 'style', 'as', "Johnson's", '.']]
text: The new world of English words came out in 1658 and  a dictionary of 40,000 words had been prepared in 1721 by Nathan Bailey  , though none was as comprehensive in breadth or style as Johnson's  .
        0   1     2  3       4     5    6   7  8    9  10 11         12 13     14    15  16   17 

In [10]:
for i, sample_line in enumerate(sample_lines):

    text = sample_line['text'].lower()
    text = remove_accents_from_string(text)
    text_tokens = text.split()

    doc = nlp([text_tokens])

    print(f'--- Sentence {i} ---')
    print(*[f'word: {word.text+" "}\tlemma: {word.lemma}' for sent in doc.sentences for word in sent.words], sep='\n')


--- Sentence 0 ---
word: in 	lemma: in
word: hinduism 	lemma: hinduism
word: , 	lemma: ,
word: the 	lemma: the
word: 60th 	lemma: 100th
word: birthday 	lemma: birthday
word: of 	lemma: of
word: a 	lemma: a
word: man 	lemma: man
word: is 	lemma: be
word: called 	lemma: call
word: sashti 	lemma: sashti
word: poorthi 	lemma: poorthi
word: . 	lemma: .
--- Sentence 1 ---
word: the 	lemma: the
word: new 	lemma: new
word: world 	lemma: world
word: of 	lemma: of
word: english 	lemma: English
word: words 	lemma: word
word: came 	lemma: come
word: out 	lemma: out
word: in 	lemma: in
word: 1658 	lemma: 1658
word: and 	lemma: and
word: a 	lemma: a
word: dictionary 	lemma: dictionary
word: of 	lemma: of
word: 40,000 	lemma: 40000
word: words 	lemma: word
word: had 	lemma: have
word: been 	lemma: be
word: prepared 	lemma: prepare
word: in 	lemma: in
word: 1721 	lemma: 1721
word: by 	lemma: by
word: nathan 	lemma: nathan
word: bailey 	lemma: bailly
word: , 	lemma: ,
word: though 	lemma: though
word: 

Now we have another problem: sometimes the index in the annotations is off. We need to account for that as in some cases (e.g. in the second sample line) the word we need to replace will remain along with its explicit senses, that replaced an unrelated token instead.

In [11]:
def jaccard_similarity(set1, set2):
    intersection = len(set1.intersection(set2))
    union = len(set1.union(set2))
    similarity = intersection / union if union != 0 else 0  # Avoid division by zero
    return similarity

def find_word_in_window(lemmatized_tokens, window_span, sense_key, use_jaccard_similarity=0.6, window_size=1):
    """
    We have a few tokens and a sense_key, that contains the lemma of a word inside the text.
    We want to find the index of that word.
    """

    start_idx = max(window_span[0] - window_size, 0)
    end_idx = min(window_span[1] + window_size, len(lemmatized_tokens))

    lemma_from_sense_key = sense_key[:sense_key.index('%')]

    lemma_scores = dict()
    for index in range(start_idx, end_idx):
        lemma = lemmatized_tokens[index]
        # print(f'Token [{token}] vs lemma [{lemma}]')

        lemma_scores[index] = jaccard_similarity(set(lemma), set(lemma_from_sense_key)) if use_jaccard_similarity > 0 else lemma == lemma_from_sense_key
    
    # print(lemma_scores)

    max_index = max(lemma_scores, key=lemma_scores.get)
    max_score = lemma_scores[max_index]

    if max_score < use_jaccard_similarity:
        raise ValueError(f'No match in window for token {sense_key} in window {lemmatized_tokens[start_idx:end_idx + 1]}')
    return max_index

In [12]:
for sample_line in sample_lines:

    pretty_print(sample_line, print_indexes=True)

    text = sample_line['text'].lower()
    text = remove_accents_from_string(text)
    text_tokens = text.split()

    doc = nlp([text_tokens])    

    lemmatized_tokens = [word.lemma for sent in doc.sentences for word in sent.words]


    for annotation in sample_line['annotations']:
        window_start = max(annotation['token_span'][0], 0)
        window_end = min(annotation['token_span'][1], len(lemmatized_tokens))
        idx = find_word_in_window(lemmatized_tokens, (window_start, window_end), annotation['label'], use_jaccard_similarity=0.6, window_size=4)
        print(f'{annotation["label"]} in range ({window_start} - {window_end}) -> true index = {idx}')
    
    print()

text: In Hinduism , the 60th birthday of a man is called Sashti poorthi  .
       0        1 2   3    4        5  6 7   8  9     10     11      12 13
annotations: {'token_span': [5, 6], 'label': 'birthday%1:28:00::'}
             {'token_span': [8, 9], 'label': 'man%1:18:00::'}
birthday%1:28:00:: in range (5 - 6) -> true index = 5
man%1:18:00:: in range (8 - 9) -> true index = 8

text: The new world of English words came out in 1658 and  a dictionary of 40,000 words had been prepared in 1721 by Nathan Bailey  , though none was as comprehensive in breadth or style as Johnson's  .
        0   1     2  3       4     5    6   7  8    9  10 11         12 13     14    15  16   17       18 19   20 21     22     23 24     25   26  27 28            29 30      31 32    33 34        35 36
annotations: {'token_span': [1, 2], 'label': 'world%1:14:01::'}
             {'token_span': [4, 5], 'label': 'word%1:10:00::'}
             {'token_span': [11, 12], 'label': 'dictionary%1:10:00::'}
             

We can build the final preprocessing routine

In [19]:
def preprocess_line(line, 
                    model,
                    stopwords=[], 
                    punctuation=[],
                    lemmatize=True,
                    filtering_regex=None, 
                    replace_with_sense_key=True,
                    remove_accents=True,
                    use_jaccard_similarity=0.6,
                    window_size = 2,
                    produce_both=False
                    ):


    if stopwords is None:
        stopwords = []

    if punctuation is None:
        punctuation = []

    text = line['text'].lower()
    if remove_accents:
        text = remove_accents_from_string(text)

    text_split = text.lower().split()
    doc = model([text_split])
    tokens = [word for sent in doc.sentences for word in sent.words]

    annotations = line['annotations']

    # Indexes of the words we may want to replace with a sense key
    # {token1_index: token1, ...}
    annotations_data = {(annotation['token_span'][0], annotation['token_span'][1]): annotation['label'] for annotation in annotations}
    annotation_indexes = {find_word_in_window(
        [token.lemma for token in tokens], 
        (k[0], k[1]), 
        v, 
        use_jaccard_similarity=use_jaccard_similarity,
        window_size=window_size
    ): v for k, v in annotations_data.items()}
    
    # This will be then joined to create a new preprocessed sentence
    new_tokens = []
    new_semantic_tokens = []
    
    # For each token, check if it is a stopword, punctuation, if it matches the filter regex 
    # and if it needs to be retained or replaced with its sense key. Doc now tokenizes as the
    # standard string split, so there will be no differences in the token indexes
    for i, token in enumerate(tokens):

        token_text = token.lemma if lemmatize else token.text
        
        # The token is a word we can replace with the lemma key
        if i in annotation_indexes.keys():

            # We may want to produce both sentences, one with lemma key and one without
            # in order to train a model for word embeddings and another model with
            # sense embeddings
            if produce_both:
                new_tokens.append(token_text)
                new_semantic_tokens.append(annotation_indexes[i]) # [i][1])
            else:
                if replace_with_sense_key:
                    # print(f'Token {token_text:20s} is good, we keep it as {annotation_indexes[i][1]}')
                    new_tokens.append(annotation_indexes[i]) # [i][1])
                else:
                    # print(f'Token {token_text:20s} is good, we keep it as {token_text}')
                    new_tokens.append(token_text)
                
        # The token is a stopword
        elif token_text in stopwords:
            # print(f'Token {token_text:20s} is a stopword')
            continue

        # The token is punctuation
        elif token_text in punctuation:
            # print(f'Token {token_text:20s} is punctuation')
            continue

        # The token matches the regex, it is not valid
        elif filtering_regex is not None and filtering_regex.search(token_text):
            # print(f'Token {token_text:20s} is filtered out from regex')
            continue

        # The token is valid, we can retain it
        else:
            # print(f'Token {token_text:20s} is good, we keep it')
            new_tokens.append(token_text)
            if produce_both:
                new_semantic_tokens.append(token_text)

    return new_tokens, new_semantic_tokens


In [14]:
# Download stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

# Regex to discard numbers, special characters and punctuation
regexp_alphbetic = re.compile('[^a-zA-Z-]+')

[nltk_data] Downloading package stopwords to /home/daniel/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [15]:
for sample_line in sample_lines:

    text = sample_line['text']

    print('Data before: ')
    print(text)

    text_split = text.lower().split()
    print(text_split)

    doc = nlp([text_split])
    tokens = [word.lemma for sent in doc.sentences for word in sent.words]
    print('Lemmatized tokens:')
    print(tokens)

    try:
        print('Preprocessed data WITHOUT jaccard similarity:')
        print(" ".join(preprocess_line(sample_line, nlp, stopwords=stop_words, filtering_regex=regexp_alphbetic, use_jaccard_similarity=1)[0]))
    except:
        print('Exception')

    print('Preprocessed data WITH jaccard similarity:')
    print(" ".join(preprocess_line(sample_line, nlp, stopwords=stop_words, filtering_regex=regexp_alphbetic)[0]))

    print()

Data before: 
In Hinduism , the 60th birthday of a man is called Sashti poorthi .
['in', 'hinduism', ',', 'the', '60th', 'birthday', 'of', 'a', 'man', 'is', 'called', 'sashti', 'poorthi', '.']
Lemmatized tokens:
['in', 'hinduism', ',', 'the', '100th', 'birthday', 'of', 'a', 'man', 'be', 'call', 'sashti', 'poorthi', '.']
Preprocessed data WITHOUT jaccard similarity:
hinduism birthday%1:28:00:: man%1:18:00:: call sashti poorthi
Preprocessed data WITH jaccard similarity:
hinduism birthday%1:28:00:: man%1:18:00:: call sashti poorthi

Data before: 
The new world of English words came out in 1658 and a dictionary of 40,000 words had been prepared in 1721 by Nathan Bailey , though none was as comprehensive in breadth or style as Johnson's .
['the', 'new', 'world', 'of', 'english', 'words', 'came', 'out', 'in', '1658', 'and', 'a', 'dictionary', 'of', '40,000', 'words', 'had', 'been', 'prepared', 'in', '1721', 'by', 'nathan', 'bailey', ',', 'though', 'none', 'was', 'as', 'comprehensive', 'in', 

# Actual preprocessing

In [23]:
# Create the actual datasets 
original_sentences = []
non_semantic_tokens = []
semantic_tokens = []
discarded_count = 0

pbar = tqdm(lines)
for line in pbar:
    
    try:
        
        non_semantic, semantic = preprocess_line(
            line, 
            nlp, 
            stopwords=stop_words, 
            filtering_regex=regexp_alphbetic, 
            produce_both=True, 
            use_jaccard_similarity=0.6
        )

        original_sentences.append(line['text'])
        non_semantic_tokens.append(non_semantic)
        semantic_tokens.append(semantic)

        pbar.set_postfix(discarded=discarded_count, refresh=False)

    except:
        # print('Discarded: ')
        # pretty_print(line, print_indexes=True)
        # print()
        discarded_count += 1

print(f'Original sentence                : {original_sentences[0]}')
print(f'Preprocessed sentence            : {" ".join(non_semantic_tokens[0])}')
print(f'Preprocessed sentence with senses: {" ".join(semantic_tokens[0])}')
print(f'Discarded {discarded_count} out of 500000')

100%|██████████| 534300/534300 [1:39:35<00:00, 89.41it/s, discarded=19706]  

Original sentence                : In Hinduism , the 60th birthday of a man is called Sashti poorthi .
Preprocessed sentence            : hinduism birthday man call sashti poorthi
Preprocessed sentence with senses: hinduism birthday%1:28:00:: man%1:18:00:: call sashti poorthi
Discarded 19706 out of 500000


Now despite discarding some sentences at least we can rest assured that only senses for the specified tokens appear in the dataset (before we could've had "word", "word%1:10:00" and so on and this would've affected the similarity score)

We could filter out uncommon words that have a low frequency but we can do the same specifying the `min_count` parameter of the Word2Vec model

In [ ]:
def filter_frequency(sentences, frequency):

    ## remove words that appear only once
    frequency_dict = defaultdict(int)
    for sentence in tqdm(sentences):
        # tokens = sentence.split()
        # for token in tokens:
        for token in sentence:
            frequency_dict[token] += 1

    texts = [[token for token in sentence.split() if frequency_dict[token] > frequency]
            for sentence in sentences]

    return texts

In [ ]:
# ff_word_sentences = filter_frequency(word_sentences, 1)
# ff_sense_sentences = filter_frequency(sense_sentences, 1)

# for i in range(10):
#   print(ff_word_sentences[i])

Save the preprocessed dictionary to file

In [24]:
# Specify the output JSONLines file. Set it to None to skip saving the dataset
output_file = 'preprocessed_dataset_v2_stanza.jsonl'
# output_file = None

if output_file is not None:

    # Open the JSONLines file for writing
    with jsonlines.open(output_file, mode='w') as writer:

        for original_sentence, non_semantic_tokens_sentence, semantic_tokens_sentence in tqdm(zip(original_sentences, non_semantic_tokens, semantic_tokens)):

            # Create a dictionary for each set of strings
            data = {
                'original': original_sentence,
                'non_semantic': non_semantic_tokens_sentence,
                'semantic': semantic_tokens_sentence
            }
            
            # Write the dictionary to the JSONLines file
            writer.write(data)

514594it [00:05, 98033.62it/s] 
